In [8]:
from sage.all import *
import json
from Crypto.Util.number import long_to_bytes

data = json.load(open('dist-meow-log-meow-log-meow-e/chall.json'))
n, e, c1, c2, po = data['n'], data['e'], data['c1'], data['c2'], data['po']

In [2]:
def e_power(x, field, terms):
    """
    Compute e^x in a finite field using Taylor series approximation.
    
    Args:
        x: element in the finite field
        field: the finite field (e.g., GF(p) or GF(p^n))
        terms: number of Taylor series terms to compute
    
    Returns:
        approximation of e^x in the finite field as 1 + x + x^2/2! + x^3/3! + ... + x^(terms-1)/(terms-1)!
    """
    result = field(1)  # Start with e^0 = 1
    x_power = field(1)  # x^0 = 1
    factorial = 1
    
    for n in range(1, terms):
        x_power *= x  # x^n
        factorial *= n  # n!
        
        # In finite fields, we need the multiplicative inverse of factorial
        try:
            factorial_inv = field(factorial)^(-1)
            term = x_power * factorial_inv
            result += term
        except ZeroDivisionError:
            # If factorial is not invertible (divisible by char(field)), skip this term
            continue
    
    return result

In [3]:
def naive_gcd(a, b):
    """
    Compute the GCD of two polynomials using the naive method
    """
    while b != 0:
        a, b = b, a % b
    return a


In [4]:
def hgcd(a0, a1, n):
    """
    Half GCD (HGCD) algorithm for polynomials
    
    Sources:
    (1) Algorithm: https://web.archive.org/web/20210512230349/https://web.cs.iastate.edu/~cs577/handouts/polydivide.pdf
    (2) rkm0959's implementation: https://github.com/rkm0959/Implementations/blob/main/Half_GCD/code.sage
    """

    P.<x> = PolynomialRing(Zmod(n))

    # Base case: if deg(a1) ≤ deg(a0)/2, return identity matrix
    if a1.degree() <= a0.degree()/2 or a0.degree() == 1:
        return matrix(P, [[1, 0], [0, 1]])
    
    # Calculate m = floor(deg(a0)/2)
    m = a0.degree() // 2
    
    b0, c0 = a0.quo_rem(x^m)  # b0 = coefficients of x^m and higher terms
    b1, c1 = a1.quo_rem(x^m)  # b1 = coefficients of x^m and higher terms
    
    # Recursive call on quotients
    R = hgcd(b0, b1, n)
    
    # Apply the transformation matrix to [a0, a1]
    d, e = R * vector([a0, a1])
    
    q, r = d.quo_rem(e)
    
    # Calculate floor(m/2) for the next recursion
    xm2 = x ^ (m // 2)
    
    # Split e and f at degree floor(m/2)
    g0, h0 = e.quo_rem(xm2)  # g0 = coefficients of x^[m/2] and higher terms
    g1, h1 = r.quo_rem(xm2)  # g1 = coefficients of x^[m/2] and higher terms
    
    # Recursive call on quotients
    S = hgcd(g0, g1, n)
    
    # Return the combined transformation matrix
    return S * matrix([[0, 1], [1, -q]]) * R


def poly_gcd(a0, a1, n):
    """
    GCD calculation for polynomials using HGCD
    
    Parameters:
    - a0, a1: polynomials in x
    
    Returns:
    - GCD of a0 and a1 (monic)
    """
    # Ensure a0 has higher or equal degree than a1
    if a0.degree() < a1.degree():
        a0, a1 = a1, a0
    
    # Base case: if a1 divides a0, return a1 (made monic)
    if a0 % a1 == 0:
        return a1.monic()
    
    # Recursive case: use HGCD to speed up the calculation
    R = hgcd(a0, a1, n)
    b0, b1 = R * vector([a0, a1])
    
    # Check if b1 divides b0
    if b0 % b1 == 0:
        return b1.monic()
    
    # Continue with standard Euclidean algorithm
    c = b0 % b1
    return poly_gcd(b1, c, n)

In [5]:
# Convert to polynomial ring
F = Zmod(n)
R.<x> = PolynomialRing(F)

# Create polynomials from the ciphertexts
a0 = x^e - c1
a1 = e_power(x, F, po)^e - c2

In [ ]:
gcd = poly_gcd(a0, a1, n)
# gcd = naive_gcd(a0, a1)
gcd = gcd.monic()

print(gcd)
assert gcd.degree() == 1, "GCD is not linear"

if gcd.degree() == 1:
    print(f"Found monic linear polynomial at degree {po}: {gcd[0]}")
    m = int(-gcd[0] % n)
    print(f"Message (m): {long_to_bytes(m)}")

x + 773094481469452914191820180355932888137411285296201681675647238679182811867016799759752294444602335484001342007091261891167739829179710713033743377713086928858838321238203250127944644442180494166554257140640287666187358232021768574431991525121210520816528107160708263424123954527497753674923849353304523242596408140495576695896537430880132141760596444138848195681561853320074540387048874010002749429119440078121814333551381659545557952573206646347492523210590610136042291472995418637260769973114084442166291491817806972264557417529697859636556105810693043784041062070584218846408479539008172817743913111935229663624164589302463953568329978672321053744058827357861853717623640488830867496322710276759686012168422031324473498094565013413027511804003071117650333618513006072310251194458490624116206774267317730759402362505240393480261651785520635025017684722608686665030671070919613381332381635684454578434832466239954677639314073776667266961688069746502404275799195096227969140687387036072497136246

NameError: name 'long_to_bytes' is not defined

In [12]:
print(f"Message (m): {long_to_bytes(m)}")

Message (m): b'grey{me0w_m3ow_s0lUtioN_t0o_Sl0w_4_m3-oW!!!'
